In [1]:
from crawl4ai import AsyncWebCrawler, CrawlerRunConfig, CacheMode
import asyncio

urls_to_crawl = [
    "https://docs.llamaindex.ai/en/stable/understanding/",
]

config = CrawlerRunConfig(
    cache_mode=CacheMode.BYPASS,
    page_timeout=80000,
    word_count_threshold=50,
)

async def crawl_website():
    data_res = {"data": []}

    async with AsyncWebCrawler(headless=True, verbose=True) as crawler:
        results = await crawler.arun_many(urls_to_crawl, config=config)

        for result in results:
            if result.success:
                title = result.metadata.get("title", "")
                if not title and result.markdown:
                    lines = result.markdown.raw_markdown.split("\n")
                    for line in lines:
                        if line.startswith("#"):
                            title = line.strip("#").strip()
                            break

                data_res["data"].append(
                    {
                        "text": result.markdown.raw_markdown if result.markdown else "",
                        "meta": {"url": result.url, "meta": {"title": title}},
                    }
                )

    return data_res

# Create new event loop for Jupyter
loop = asyncio.new_event_loop()
asyncio.set_event_loop(loop)
data_res = loop.run_until_complete(crawl_website())
loop.close()

print(data_res)

RuntimeError: Cannot run the event loop while another loop is running